In [1]:
import os  # OS 경로/폴더 생성 등 시스템 기능 사용
import torch  # PyTorch 메인 패키지
import torch.nn as nn  # 신경망 모듈(레이어, 손실 등)
from sklearn.model_selection import StratifiedKFold  # 계층적 K-Fold 분할
from tqdm import tqdm  # 진행률 표시 바
from torchvision import models, transforms  # 사전학습 모델/이미지 변환
from torch.utils.data import Dataset, DataLoader  # PyTorch 데이터셋/로더
from datasets import load_dataset, ClassLabel, DatasetDict  # Hugging Face Datasets 유틸
import numpy as np  # 수치 계산
from torch.amp import autocast, GradScaler  # 혼합정밀 자동 캐스트/스케일러
import math  # 수학 유틸
import time  # 시간 측정
from typing import Optional, Tuple, Dict  # 타입 힌트
import torch.nn.functional as F  # 함수형 API (loss/activation 등)
from sklearn.metrics import f1_score, classification_report, confusion_matrix, balanced_accuracy_score, top_k_accuracy_score  # 평가 지표
from datasets import load_from_disk  # HF dataset 디스크 로드(현재 코드에선 미사용)
from torch.autograd import Variable  # 오토그라드 Variable(현 PyTorch에선 텐서와 동일, 미사용)
from collections import Counter  # 라벨 카운트

import json, os  # JSON 입출력/OS 유틸(중복 import but harmless)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 사용 디바이스 선택(CUDA 우선)

train_transform = transforms.Compose([  # 학습용 이미지 증강/전처리 파이프라인
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(3/4, 4/3)),  # 랜덤 크롭+리사이즈
    transforms.RandomHorizontalFlip(p=0.5),  # 좌우 반전
    transforms.RandomRotation(degrees=15),  # 회전
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.15, hue=0.02),  # 색상/밝기/대비 변형
    transforms.ToTensor(),  # 텐서 변환(CHW, [0,1])
    transforms.Normalize([0.485,0.456,0.406],  # ImageNet 평균/표준편차 정규화
                         [0.229,0.224,0.225]),
    transforms.RandomErasing(p=0.25, scale=(0.02, 0.1), ratio=(0.3, 3.3))  # 랜덤 지우기(regularization)
])

val_transform = transforms.Compose([  # 검증용(약한) 전처리
    transforms.Resize(256),  # 리사이즈
    transforms.CenterCrop(224),  # 중앙 크롭
    transforms.ToTensor(),  # 텐서 변환
    transforms.Normalize([0.485,0.456,0.406],  # 정규화
                         [0.229,0.224,0.225])
])


def prepare_dataset():  # 데이터셋 로드/클린업/스플릿/라벨정리 함수
    dataset = load_dataset("Densu341/Fresh-rotten-fruit")  # HF 허브에서 데이터 로드

    # 1) 라벨 제거
    remove_labels = [18, 20, 16, 13, 2, 5, 7, 9]  # 제거할 라벨 인덱스 목록
    labels = np.array(dataset["train"]["label"])  # 원본 라벨 배열
    mask = ~np.isin(labels, remove_labels)  # 제거할 라벨 제외 마스크
    clean = dataset["train"].select(np.where(mask)[0])  # 필터링된 서브셋

    # 2) split (결정적)
    split = clean.train_test_split(test_size=0.2, seed=42)  # train/val 80:20 고정 분할
    train_ds, val_ds = split["train"], split["test"]  # 분할 결과

    # 3) 라벨 재매핑 (이름 기준으로 0..C-1)
    uniq = sorted(set(train_ds["label"]) | set(val_ds["label"]))  # 사용되는 라벨 인덱스 집합
    names = [train_ds.features["label"].int2str(i) for i in uniq]  # 라벨 이름 리스트
    new_lbl = ClassLabel(num_classes=len(names), names=names)  # 새 ClassLabel 생성

    def remap(example):  # 라벨 인덱스를 새 인덱스로 재매핑
        name = train_ds.features["label"].int2str(example["label"])  # 기존 인덱스→이름
        example["label"] = names.index(name)  # 이름→새 인덱스
        return example

    train_ds = train_ds.map(remap, num_proc=os.cpu_count()//2,  # 병렬 맵 적용(캐시 사용)
                            load_from_cache_file=True, desc="Remap train")
    val_ds   = val_ds.map(remap,   num_proc=os.cpu_count()//2,
                            load_from_cache_file=True, desc="Remap val")

    train_ds = train_ds.cast_column("label", new_lbl)  # 라벨 스키마를 새 ClassLabel로 캐스팅
    val_ds   = val_ds.cast_column("label",  new_lbl)

    # 4) RGB 통일 (결정적)
    def to_rgb(example):  # 비-RGB 이미지를 RGB로 변환
        img = example["image"]  # PIL 이미지
        if img.mode != "RGB":  # 모드가 RGB가 아니면
            img = img.convert("RGB")  # RGB 변환
        example["image"] = img  # 교체
        return example

    train_ds = train_ds.map(to_rgb, num_proc=os.cpu_count()//2,  # RGB 통일(train)
                            load_from_cache_file=True, desc="RGB train")
    val_ds   = val_ds.map(to_rgb,   num_proc=os.cpu_count()//2,  # RGB 통일(val)
                            load_from_cache_file=True, desc="RGB val")

    # ✅ 여기서 끝! set_transform 안 씀
    return DatasetDict({"train": train_ds, "test": val_ds})  # 최종 DatasetDict 반환

In [ ]:
# PyTorch Dataset 래퍼
# --------------------------------------------------
class FruitHFDataset(Dataset):  # HF Dataset을 PyTorch Dataset으로 감싸는 래퍼
    def __init__(self, hf_dataset, transform=None):  # HF dataset과 transform 주입
        self.ds = hf_dataset  # 내부 참조
        self.tf = transform  # 변환 파이프라인

    def __len__(self):  # 전체 샘플 수
        return len(self.ds)

    def __getitem__(self, idx):  # 인덱스로 샘플 접근
        item = self.ds[idx]            # image: PIL.Image, label: int
        img  = item["image"]  # 이미지 추출
        if self.tf is not None:  # 변환이 설정되어 있으면
            img = self.tf(img)         # Tensor(C,H,W)로 변환/증강
        label = item["label"]  # 라벨 추출
        # long 보장
        import torch  # 지역 import (원본 코드 유지)
        if not torch.is_tensor(label):  # 라벨이 텐서가 아니면
            import torch  # 지역 import (원본 코드 유지)
            label = torch.tensor(label, dtype=torch.long)  # LongTensor로 변환
        else:
            label = label.to(dtype=torch.long)  # dtype 보정
        return img, label  # (이미지 텐서, 라벨 텐서) 반환

In [ ]:
# 모델 코드 cmt기반 cnn+t     
# ------------------------------------------------------------
class DropPath(nn.Module):  # Stochastic Depth 구현(샘플별 경로 드롭)
    """ per-sample DropPath (Stochastic Depth) """
    def __init__(self, drop_prob: float = 0.0):  # 드롭 확률
        super().__init__()
        self.drop_prob = float(drop_prob)  # float로 보관

    def forward(self, x):  # 전방 계산
        if self.drop_prob == 0.0 or not self.training:  # 학습 아닐 때/확률 0이면 패스
            return x
        keep_prob = 1.0 - self.drop_prob  # 유지 확률
        # x: (B, ...)
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # 배치 차원만 난수 생성
        random_tensor = x.new_empty(shape).bernoulli_(keep_prob)  # 베르누이 샘플
        return x.div(keep_prob) * random_tensor  # 스케일 보정 후 마스킹

class DepthwiseConv2d(nn.Module):  # 채널별(depthwise) 컨볼루션
    def __init__(self, channels, k=3, s=1, p=1, bias=False):  # 하이퍼파라미터
        super().__init__()
        self.dw = nn.Conv2d(channels, channels, k, s, p, groups=channels, bias=bias)  # groups=channels

    def forward(self, x):  # 전방 계산
        return self.dw(x)

class ConvBNGELU(nn.Module):  # Conv→BN→GELU 블록
    def __init__(self, in_ch, out_ch, k=3, s=1, p=1):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, k, s, p, bias=False)  # 합성곱
        self.bn   = nn.BatchNorm2d(out_ch, eps=1e-5, momentum=0.1)  # 배치정규화
        self.act  = nn.GELU()  # 활성화

    def forward(self, x):  # 전방 계산
        return self.act(self.bn(self.conv(x)))  # Conv→BN→GELU 순서

class ConvStage(nn.Module):  # 다운샘플 포함한 Conv 스테이지
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.ds   = ConvBNGELU(in_ch, out_ch, k=3, s=2, p=1)   # s=2로 다운샘플
        self.body = ConvBNGELU(out_ch, out_ch, k=3, s=1, p=1)  # 본체 블록

    def forward(self, x):  # 전방 계산
        x = self.ds(x)  # 다운샘플
        x = self.body(x)  # 특징 추출
        return x
    
class LPU(nn.Module):  # Local Perception Unit: 지역 DWConv+BN+GELU
    """ Local Perception Unit: 3x3 depthwise → GELU → BN (채널 보존) """
    def __init__(self, channels):
        super().__init__()
        self.dw  = DepthwiseConv2d(channels, k=3, s=1, p=1, bias=False)  # 깊이별 합성곱
        self.bn  = nn.BatchNorm2d(channels, eps=1e-5, momentum=0.1)  # BN
        self.act = nn.GELU()  # GELU

    def forward(self, x):  # 전방 계산
        x = self.dw(x)  # DWConv
        x = self.bn(x)  # 정규화
        x = self.act(x)  # 활성화
        return x
    
    
class MLP(nn.Module):  # Transformer FFN 유사 MLP
    def __init__(self, dim, mlp_ratio=3.0, drop=0.1):
        super().__init__()
        hidden = int(dim * mlp_ratio)  # 숨겨진 차원 계산
        self.fc1 = nn.Linear(dim, hidden)  # 첫 FC
        self.act = nn.GELU()  # 활성화
        self.fc2 = nn.Linear(hidden, dim)  # 복귀 FC
        self.drop = nn.Dropout(drop)  # 드롭아웃

    def forward(self, x):  # 전방 계산
        x = self.fc1(x); x = self.act(x); x = self.drop(x)  # FC1→활성→드롭
        x = self.fc2(x); x = self.drop(x)  # FC2→드롭
        return x

class TransformerBlock(nn.Module):  # MSA+MLP 블록(LN/DropPath 포함)
    def __init__(self, dim, num_heads, mlp_ratio=3.0, attn_drop=0.0, proj_drop=0.1, drop_path=0.0): #drop_path는 에포크 늘어나면 늘리삼
        super().__init__()
        self.norm1 = nn.LayerNorm(dim, eps=1e-6) #1e-5 ~ 1e-7 사이 값  # 어텐션 전 정규화
        self.attn  = nn.MultiheadAttention(dim, num_heads, dropout=attn_drop, batch_first=True)  # 멀티헤드 어텐션
        self.drop1 = nn.Dropout(proj_drop)  # 어텐션 출력 드롭아웃
        self.dp1   = DropPath(drop_path)  # 스토캐스틱 깊이

        self.norm2 = nn.LayerNorm(dim, eps=1e-6)  # MLP 전 정규화
        self.mlp   = MLP(dim, mlp_ratio=mlp_ratio, drop=proj_drop)  # FFN
        self.dp2   = DropPath(drop_path)  # 스토캐스틱 깊이

    def forward(self, x):  # 전방 계산
        # x: (B, N, C)
        y, _ = self.attn(self.norm1(x), self.norm1(x), self.norm1(x), need_weights=False)  # self-attention
        x = x + self.dp1(self.drop1(y))  # 잔차 연결 + 드롭경로
        x = x + self.dp2(self.mlp(self.norm2(x)))  # LN→MLP→잔차
        return x

    

class CMTClassifier(nn.Module):  # CNN+Transformer 결합 분류기
    def __init__(
        self,
        num_classes: int,  # 클래스 수
        # CNN channels
        stem_channels: int = 64,  # stem 출력 채널
        c_stage1: int = 96,  # stage1 채널
        c_stage2: int = 128,  # stage2 채널
        c_stage3: int = 160,  # stage3 채널
        # Transformer dims/heads/depths
        t_dim1: int = 256,  t_heads1: int = 4,  t_depth1: int = 3,  t_mlp1: float = 3.0,  # 14x14 스테이지 설정
        t_dim2: int = 384,  t_heads2: int = 6,  t_depth2: int = 6,  t_mlp2: float = 3.5,  # 7x7 스테이지 설정
        attn_drop: float = 0.0,  # 어텐션 드롭아웃
        proj_drop: float = 0.1,  # 프로젝션 드롭아웃
        drop_path_rate: float = 0.0,  # 에폭 짧으면 0.0, 길게 학습하면 0.05~0.1  # 스토캐스틱 깊이 범위
    ):
        super().__init__()
        self.num_classes = num_classes  # 클래스 수 저장

        # ----- CNN stem (224 -> 112) -----
        self.stem = nn.Sequential(  # 초반 다운샘플/특징 추출
            ConvBNGELU(3, stem_channels // 2, k=3, s=2, p=1),  # s=2로 크기 절반
            ConvBNGELU(stem_channels // 2, stem_channels, k=3, s=1, p=1),  # 채널 확장
        )

        # ----- CNN stages (112 -> 56 -> 28 -> 14) -----
        self.stage1 = ConvStage(stem_channels, c_stage1)  # 112→56
        self.stage2 = ConvStage(c_stage1, c_stage2)  # 56→28
        self.stage3 = ConvStage(c_stage2, c_stage3)  # 28→14

        # ----- to embed (14x14, C3 -> D1) -----
        self.to_embed1 = nn.Conv2d(c_stage3, t_dim1, kernel_size=1, stride=1, padding=0, bias=True)  # 채널→임베딩

        # ----- Stage A @14x14 : LPU → Transformer(depth=t_depth1) -----
        self.lpu1 = LPU(t_dim1)  # 지역 인식 유닛
        dpr1 = torch.linspace(0, drop_path_rate * 0.5, steps=t_depth1).tolist()  # 드롭패스 스케줄
        self.trans1 = nn.Sequential(*[  # 14x14에서 Transformer 블록 스택
            TransformerBlock(
                dim=t_dim1, num_heads=t_heads1, mlp_ratio=t_mlp1,
                attn_drop=attn_drop, proj_drop=proj_drop, drop_path=dpr1[i]
            ) for i in range(t_depth1)
        ])

        # ----- down tokens: 14->7, D1->D2 -----
        self.down_tokens = nn.Sequential(  # 토큰 맵 다운샘플(해상도↓, 채널↑)
            nn.Conv2d(t_dim1, t_dim2, kernel_size=3, stride=2, padding=1, bias=False),  # s=2
            nn.BatchNorm2d(t_dim2, eps=1e-5, momentum=0.1),  # BN
            nn.GELU(),  # 활성화
        )

        # ----- Stage B @7x7 : LPU → Transformer(depth=t_depth2) -----
        self.lpu2 = LPU(t_dim2)  # 7x7 단계 LPU
        dpr2 = torch.linspace(drop_path_rate * 0.5, drop_path_rate, steps=t_depth2).tolist()  # 드롭패스 스케줄 상반부
        self.trans2 = nn.Sequential(*[  # 7x7에서 Transformer 블록 스택
            TransformerBlock(
                dim=t_dim2, num_heads=t_heads2, mlp_ratio=t_mlp2,
                attn_drop=attn_drop, proj_drop=proj_drop, drop_path=dpr2[i]
            ) for i in range(t_depth2)
        ])

        # ----- Head -----
        self.head_norm = nn.LayerNorm(t_dim2, eps=1e-6)  # 최종 LN
        self.fc        = nn.Linear(t_dim2, num_classes)  # 분류기 FC

        self.apply(self._init_weights)  # 가중치 초기화 적용

    def _init_weights(self, m):  # 모듈별 초기화 방식 정의
        if isinstance(m, nn.Linear):  # 선형층
            nn.init.trunc_normal_(m.weight, std=0.02)  # Truncated normal
            if m.bias is not None: nn.init.constant_(m.bias, 0)  # bias 0
        elif isinstance(m, nn.Conv2d):  # 합성곱
            nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")  # He 초기화
            if m.bias is not None: nn.init.constant_(m.bias, 0)  # bias 0
        elif isinstance(m, (nn.LayerNorm, nn.BatchNorm2d)):  # 정규화 계층
            if hasattr(m, "weight") and m.weight is not None: nn.init.ones_(m.weight)  # gamma=1
            if hasattr(m, "bias") and m.bias is not None:     nn.init.zeros_(m.bias)  # beta=0

    def forward(self, x):  # 전방 계산
        # CNN 얕은 특징
        x = self.stem(x)          # 224 -> 112로 다운샘플 + 특징
        x = self.stage1(x)        # 112 -> 56
        x = self.stage2(x)        # 56  -> 28
        x = self.stage3(x)        # 28  -> 14

        # 임베딩
        x = self.to_embed1(x)     # B x D1 x 14 x 14 (채널→임베딩 차원)

        # Stage A: LPU → Transformer @14x14
        x = self.lpu1(x)  # 지역 인식
        B, C, H, W = x.shape  # 텐서 차원 분해
        x = x.flatten(2).transpose(1, 2)           # B x 196 x D1 (패치 토큰화)
        x = self.trans1(x)  # 트랜스포머 처리
        x = x.transpose(1, 2).view(B, C, H, W)     # 다시 (B,C,H,W)로 복원

        # Downsample tokens: 14 -> 7
        x = self.down_tokens(x)                    # B x D2 x 7 x 7 (해상도 절반)

        # Stage B: LPU → Transformer @7x7
        x = self.lpu2(x)  # 지역 인식
        B, C, H, W = x.shape  # 차원 재할당
        x = x.flatten(2).transpose(1, 2)           # B x 49 x D2 (토큰)
        x = self.trans2(x)                         # 트랜스포머 처리

        # Head
        x = x.mean(dim=1)                          # 토큰 평균 풀링(GAP)
        x = self.head_norm(x)  # LN
        logits = self.fc(x)  # 클래스별 로짓
        return logits  # 최종 로짓 반환
# ------------------------------------------------------------    

In [ ]:
# 손실함수 클래스


class FocalLoss(nn.Module):  # 멀티클래스 Focal Loss 구현
    """
    Multi-class Focal Loss with per-class alpha.
    - inputs: logits (B, C)
    - targets: int labels (B,)
    """
    def __init__(self, alpha=None, gamma=2.0, reduction="mean", eps=1e-8):  # 하이퍼파라미터
        super().__init__()
        self.gamma = float(gamma)  # 난이도 조절 지수
        self.reduction = reduction  # 리덕션 방식
        self.eps = float(eps)  # 수치 안정성 epsilon

        if alpha is not None:  # 클래스 가중치가 주어지면
            alpha = torch.as_tensor(alpha, dtype=torch.float32)  # 텐서화
        self.register_buffer("alpha", alpha if alpha is not None else None)  # 버퍼로 등록(디바이스 이동 자동)

    def forward(self, inputs, targets):  # 입력 로짓, 정답 라벨
        # logits -> log-prob/prob
        log_probs = F.log_softmax(inputs, dim=1)   # (B, C) 로그소프트맥스
        probs     = log_probs.exp()                # (B, C) 확률

        targets = targets.long()  # 라벨 long 보장
        log_pt  = log_probs.gather(1, targets.unsqueeze(1)).squeeze(1)  # 정답 클래스 로그확률
        pt      = probs.gather(1, targets.unsqueeze(1)).squeeze(1)      # 정답 클래스 확률

        # --- 수치 안정화 ---
        pt = pt.clamp(min=self.eps, max=1. - self.eps)  # 확률 클램프
        # log_pt는 log_softmax 결과라 이미 안정적이지만, 혹시 모를 NaN 방지:
        log_pt = torch.log(pt)  # 로그 재계산

        # alpha_t
        if self.alpha is not None:  # 클래스별 가중치가 있으면
            alpha_t = self.alpha[targets]  # (B,)
        else:
            alpha_t = torch.ones_like(pt)  # 없으면 균등 가중

        # focal term
        focal = (1.0 - pt).pow(self.gamma)  # 어려운 샘플 가중↑
        loss  = -alpha_t * focal * log_pt  # Focal loss 공식

        if self.reduction == "mean":  # 평균 리덕션
            return loss.mean()
        elif self.reduction == "sum":  # 합 리덕션
            return loss.sum()
        return loss  # 리덕션 없음
# ------------------------------------------------------------

In [ ]:
# 메인
def main():  # 전체 파이프라인 실행 함수
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 디바이스 재확인
    print("device:", device)  # 디바이스 출력
    if torch.cuda.is_available():  # CUDA 사용 가능 시
        print(torch.cuda.get_device_name(0))  # GPU 이름 출력

    final_dataset = prepare_dataset()  # 데이터셋 준비(클린/스플릿/라벨정리)

    names = final_dataset["train"].features["label"].names  # 라벨 이름 리스트
    save_dir = "C:/Users/USER-PC/Desktop/deep/model_data"  # 저장 경로
    os.makedirs(save_dir, exist_ok=True)  # 디렉토리 생성
    with open(os.path.join(save_dir, "label_names.json"), "w", encoding="utf-8") as f:  # 라벨 이름 저장
        json.dump(names, f, ensure_ascii=False)  # JSON 덤프

        
    num_classes = len(final_dataset["train"].features["label"].names)  # 클래스 수 계산

    # ----- class-balanced alpha (FocalLoss용) -----
    train_labels = [int(x) for x in final_dataset["train"]["label"]]  # 학습 라벨 목록
    counts = Counter(train_labels)  # 클래스별 개수 카운트
    class_counts = [counts[i] for i in range(num_classes)]  # 순서대로 카운트 배열 생성

    beta = 0.999  # 효과적 수 수식의 베타(클래스 빈도 조정)
    effective_num = [1.0 - (beta ** c) for c in class_counts]  # 각 클래스의 유효 샘플 수
    raw_alpha = torch.tensor([(1.0 - beta) / (en if en > 0 else 1e-8)  # 클래스 가중 원시값
                              for en in effective_num], dtype=torch.float32)
    alpha = (raw_alpha / raw_alpha.sum()) * num_classes  # 평균 1로 스케일링
    print("alpha:", alpha.tolist())  # 가중치 출력


    EPOCHS = 5  # 학습 epoch 수
    BATCH_SIZE = 128  #데스크탑은 128로  # 배치 크기
    K = 3  # K-Fold 수
    best_acc = 0.0  # 전체 베스트 정확도

    # ==== 학습률/정규화 하이퍼파라미터 ====
    LR_CNN = 5e-5          # CNN(stem, stage1~3)용 learning rate
    LR_TRANS = 1e-4        # Transformer + 분류기 헤드용 learning rate
    WEIGHT_DECAY = 1e-4    # AdamW weight decay

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 디바이스 재설정(중복이지만 유지)
    print("device:", device, flush=True)  # 디바이스 출력(flush)
    print(torch.cuda.get_device_name(0))  # GPU 이름 출력

# StratifiedKFold를 위해 라벨 벡터 추출
    labels  = np.asarray(final_dataset["train"]["label"], dtype=np.int64)  # 라벨 배열(np.int64)
    indices = np.arange(len(final_dataset["train"]), dtype=np.int64)  # 인덱스 배열
    skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)  # 계층적 K-Fold 분할기

    fold_accs = []  # fold별 val acc 기록
    start_time = time.time()  # 전체 시작 시간
    num_classes = len(final_dataset["train"].features["label"].names)  # 클래스 수(재확인)

    
    for fold, (train_idx, val_idx) in enumerate(skf.split(indices, labels), 1):  # 각 폴드 반복
        print(f"\n================ Fold {fold}/{K} 시작 ================")  # 폴드 시작 로그
        fold_start = time.time()  # 폴드 시작 시간

        # --- (1) 데이터 분할 & DataLoader ---
        train_split = final_dataset["train"].select(list(train_idx))  # 학습 인덱스 선택(현재 아래에서 미활용)
        val_split   = final_dataset["train"].select(list(val_idx))  # 검증 인덱스 선택(현재 아래에서 미활용)

        train_ds = FruitHFDataset(final_dataset["train"], transform=train_transform)  # 전체 train에 학습 변환 적용
        val_ds   = FruitHFDataset(final_dataset["test"],  transform=val_transform)  # test split에 검증 변환 적용

        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  # 학습용 DataLoader
                                num_workers=6, pin_memory=True, persistent_workers=False, prefetch_factor=2)
        val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,  # 검증용 DataLoader
                                num_workers=6, pin_memory=True, persistent_workers=False, prefetch_factor=2)
        
        #병목인지 확인하는 코드임---------------------
        loader_start = time.time()  # 로더 속도 측정 시작
        for i, (x, y) in enumerate(train_loader):  # 배치 반복
            if i == 10:  # 10배치까지만 확인
                break
        print(f"첫 10 batch 로딩 시간: {time.time() - loader_start:.2f}초")  # 로딩 시간 출력
        #--------------------------------------------

        torch.backends.cudnn.benchmark = True  # 입력 크기 고정 시 Convolution 튜너 활성화

        # --- 모델/손실/옵티마이저 -----------------------------------------------------------------------
        model = CMTClassifier(num_classes).to(device)  # 모델 생성/디바이스 이동
        criterion = FocalLoss(alpha=alpha.to(device), gamma=2.0).to(device) #손실함수로 focal loss 사용함  # 손실 함수(Focal)
        # ---------------- 파라미터 그룹 설정 ----------------
    cnn_modules = [
        model.stem,
        model.stage1,
        model.stage2,
        model.stage3,
    ]

    trans_modules = [
        model.to_embed1,
        model.lpu1,
        model.trans1,
        model.down_tokens,
        model.lpu2,
        model.trans2,
        model.head_norm,
        model.fc,
    ]

    cnn_params = []
    trans_params = []

    for m in cnn_modules:
        cnn_params += list(m.parameters())

    for m in trans_modules:
        trans_params += list(m.parameters())

    optimizer = torch.optim.AdamW(
    [
        {"params": cnn_params,  "lr": LR_CNN},
        {"params": trans_params, "lr": LR_TRANS},
    ],
        weight_decay=WEIGHT_DECAY,
)
# ----------------------------------------------------사용  # 옵티마이저

    scaler = GradScaler()  # 혼합정밀 스케일러(AMP)

                
    val_acc_list = []  # 에폭별 val acc 기록
    val_loss_list = []  # 에폭별 val loss 기록
    val_f1_list   = []  # 에폭별 macro-F1 기록

        # --- Epoch 루프 ---
    for epoch in range(1, EPOCHS+1):  # 1~EPOCHS
            epoch_start = time.time()  # 에폭 시작 시간
            print(f"\n▶ Fold {fold}/{K} | Epoch {epoch}/{EPOCHS}")  # 에폭 시작 로그

            # ---- [Train] ----
            model.train()  # 학습 모드
            total, correct, loss_sum = 0, 0, 0.0  # 통계 변수 초기화
            pbar = tqdm(train_loader, desc=f"Fold {fold} Epoch {epoch} [Train]", ncols=100)  # 진행바

            for x, y in pbar:  # 배치 반복
                x = x.to(device, non_blocking=True)  # 입력 텐서 디바이스 이동
                y = y.to(device, non_blocking=True)  # 라벨 디바이스 이동

                optimizer.zero_grad(set_to_none=True)  # 그래디언트 0 초기화

                with autocast("cuda"):  # AMP 자동 캐스트(CUDA 전용)
                    out  = model(x)  # 모델 전방 계산
                    loss = criterion(out, y)  # Focal loss 계산

                # (선택) gradient clipping을 원한다면: scaler.unscale_ 후 clip
                # scaler.unscale_(optimizer)
                # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

                scaler.scale(loss).backward()  # 스케일된 backward
                scaler.step(optimizer)  # 옵티마이저 스텝(스케일 고려)
                scaler.update()  # 스케일 업데이트

                bs = x.size(0)  # 배치 크기
                loss_sum += loss.item() * bs  # 손실 누적(샘플수 가중)
                correct  += (out.argmax(1) == y).sum().item()  # 정답 카운트
                total    += bs  # 샘플 수 누적

            tr_acc  = correct / max(1, total)  # 학습 정확도
            tr_loss = loss_sum / max(1, total)  # 학습 손실 평균
            print(f"Train ▶ acc: {tr_acc:.4f} | loss: {tr_loss:.4f}")  # 학습 통계 출력
                # ---- [Validation] ----
            model.eval()  # 평가 모드
            v_total, v_correct, v_loss_sum = 0, 0, 0.0  # 검증 통계 초기화
            

            all_preds, all_labels, all_logits = [], [], []  # 예측/정답/로짓 저장 리스트
            with torch.inference_mode():  # no_grad + 추가 최적화
                 for x, y in tqdm(val_loader, desc=f"Fold {fold} Epoch {epoch} [Val]", ncols=100):  # 검증 반복
                    x = x.to(device, non_blocking=True)  # 입력 이동
                    y = y.to(device, non_blocking=True)  # 라벨 이동

                    with autocast("cuda"):  # AMP 평가
                        out  = model(x)  # 전방 계산
                        v_loss = criterion(out, y)  # 손실 계산

                    bs = x.size(0)  # 배치 크기
                    v_loss_sum += v_loss.item() * bs  # 손실 누적
                    preds = out.argmax(1)  # 예측 라벨
                    v_correct += (preds == y).sum().item()  # 정답 카운트
                    v_total   += bs  # 샘플 수 누적

                    # 🔹 예측/정답/로짓 저장 (CPU로 변환)
                    all_preds.extend(preds.detach().cpu().numpy())  # 예측 수집
                    all_labels.extend(y.detach().cpu().numpy())  # 정답 수집
                    all_logits.append(out.detach().cpu().numpy())  # 로짓 수집

            va_acc  = v_correct / max(1, v_total)  # 검증 정확도
            va_loss = v_loss_sum / max(1, v_total)  # 검증 손실 평균
            all_logits = np.concatenate(all_logits, axis=0)  # 로짓 합치기
            va_f1   = f1_score(all_labels, all_preds, average="macro")  # 매크로 F1
            va_bal  = balanced_accuracy_score(all_labels, all_preds)  # 균형 정확도

            try:
                va_top2 = top_k_accuracy_score(all_labels, all_logits, k=2, labels=np.arange(all_logits.shape[1]))  # top-2
                va_top3 = top_k_accuracy_score(all_labels, all_logits, k=3, labels=np.arange(all_logits.shape[1]))  # top-3
            except Exception:
                va_top2 = va_top3 = None  # 예외 시 None

    
            val_acc_list.append(va_acc)  # 리스트에 추가
            val_loss_list.append(va_loss)  # 리스트에 추가
            val_f1_list.append(va_f1)  # 리스트에 추가
            print(f"Val ▶ acc: {va_acc:.4f} | f1: {va_f1:.4f} | bal_acc: {va_bal:.4f} | loss: {va_loss:.4f}")  # 검증 로그
            if va_top2 is not None:
                print(f"      top-2: {va_top2:.4f} | top-3: {va_top3:.4f}")  # top-k 출력

            
            save_dir = "C:/Users/USER-PC/Desktop/deep/model_data"  # 저장 폴더
            os.makedirs(save_dir, exist_ok=True)  # 폴더 생성
            
            if va_acc > best_acc + 1e-6:  # 전체 베스트 갱신 조건(부동소수 보호)
                best_acc = va_acc  # 베스트 업데이트
                save_path = os.path.join(save_dir, f"best_model_fold{fold}.pt")  # 경로
                torch.save(model.state_dict(), save_path)  # 모델 가중치 저장
                print(f"New best model saved! (fold={fold}, acc={best_acc:.4f})")  # 저장 로그


            epoch_time = time.time() - epoch_start  # 에폭 소요 시간
            print(f"Epoch {epoch} 완료 (소요시간: {epoch_time:.2f}초)")  # 에폭 완료 로그

        # --- (4) Fold 종료 처리 (fold ‘안’) ---
    fold_time = time.time() - fold_start  # 폴드 소요 시간
    print(f"✅ Fold {fold} 완료! (소요시간: {fold_time/60:.2f}분)")  # 폴드 완료 로그
    fold_accs.append(val_acc_list)  # 폴드별 val acc 리스트 저장

    # --- (5) 전체 요약 (fold ‘바깥’) ---
    total_time = time.time() - start_time  # 총 학습 시간
    print(f"\n================ 학습종료 총 소요시간: {total_time/60:.2f}분 ================")  # 총 소요 로그

    print("\n===== K-Fold 결과 요약 =====")  # 요약 표제
    best_accs = [max(a) if isinstance(a, list) else a for a in fold_accs]  # 각 폴드 최고 성능 추출
    for i, acc in enumerate(best_accs, start=1):  # 폴드별 출력
        print(f"Fold {i:>2} | 최고 val_acc = {acc:.4f}")
    mean_acc = sum(best_accs) / len(best_accs)  # 평균 정확도
    print(f"평균 val_acc = {mean_acc:.4f}")  # 평균 출력

    torch.save(model.state_dict(), "best_model.pt")  # 마지막 모델 가중치 저장


if __name__ == "__main__":  # 엔트리 포인트
    torch.multiprocessing.set_start_method("spawn", force=True)  # 멀티프로세싱 시작 방법 지정
    main()  # 메인 실행